In [2]:
#import libraries
import pandas as pd
import sqlite3
import geopandas as gpd
import folium 

from folium.features import GeoJson, GeoJsonTooltip, GeoJsonPopup
import matplotlib.pyplot as plt

import scipy.stats as st

In [3]:
# Create a SQL connection to our SQLite database
con = sqlite3.connect('FPA_FOD_20221014.sqlite')

#df = pd.read_sql_query("SELECT * FROM Fires", con)
#pd.set_option('max_columns', None)
query = 'SELECT * FROM Fires;'  # Replace 'your_table' with your table name
df = pd.read_sql_query(query, con)

In [4]:
gdf = gpd.GeoDataFrame(df)
gdf.tail()

,OBJECTID,Shape,FOD_ID,FPA_ID,SOURCE_SYSTEM_TYPE,SOURCE_SYSTEM,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,NWCG_REPORTING_UNIT_NAME,SOURCE_REPORTING_UNIT,...,CONT_TIME,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,OWNER_DESCR,STATE,COUNTY,FIPS_CODE,FIPS_NAME
2303561,2303562,b'\x00\x01\xad\x10\x00\x00\xcc\x9a\xedDh=[\xc0...,400732978,ICS209_2020_11710294,INTERAGCY,IA-ICS209,BLM,USCOGRD,Grand Junction Field Office,COGRD,...,None,1.0,B,39.037890,-108.959500,MISSING/NOT SPECIFIED,CO,None,None,None
2303562,2303563,b'\x00\x01\xad\x10\x00\x00\xe8\x11\xda\xda1\xe...,400732979,ICS209_2020_11781527,INTERAGCY,IA-ICS209,ST/C&L,USCAMMU,Merced-Mariposa Unit,CAMMU,...,,100.0,D,37.148611,-119.503056,Private,CA,Madera,06039,Madera County
2303563,2303564,b'\x00\x01\xad\x10\x00\x00P\xf6\xa7\x9eV\x9c\\...,400732980,ICS209_2020_11815219,INTERAGCY,IA-ICS209,FS,USMTBRF,Bitterroot National Forest,MTBRF,...,None,50.0,C,46.151370,-114.442800,MISSING/NOT SPECIFIED,MT,None,None,None
2303564,2303565,b'\x00\x01\xad\x10\x00\x00\\\x87\xc8\xbbS\x07^...,400732982,ICS209_2020_11831809,INTERAGCY,IA-ICS209,FWS,USWAMCR,Mid Columbia National Wildlife Refuge Complex,WAMCR,...,1900,24892.0,G,46.670340,-120.114500,UNDEFINED FEDERAL,WA,Yakima,53077,Yakima County
2303565,2303566,b'\x00\x01\xad\x10\x00\x00P\xdb%(\x80\xc9S\xc0...,400732984,ICS209_2020_11977822,INTERAGCY,IA-ICS209,FS,USVAVAF,George Washington & Jefferson National Forests,VAVAF,...,1223,105.0,D,38.578900,-79.148450,UNDEFINED FEDERAL,VA,Rockingham,51165,Rockingham County


In [5]:
gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 2303566 entries, 0 to 2303565
Data columns (total 39 columns):
 #   Column                         Dtype  
---  ------                         -----  
 0   OBJECTID                       int64  
 1   Shape                          object 
 2   FOD_ID                         int64  
 3   FPA_ID                         object 
 4   SOURCE_SYSTEM_TYPE             object 
 5   SOURCE_SYSTEM                  object 
 6   NWCG_REPORTING_AGENCY          object 
 7   NWCG_REPORTING_UNIT_ID         object 
 8   NWCG_REPORTING_UNIT_NAME       object 
 9   SOURCE_REPORTING_UNIT          object 
 10  SOURCE_REPORTING_UNIT_NAME     object 
 11  LOCAL_FIRE_REPORT_ID           object 
 12  LOCAL_INCIDENT_ID              object 
 13  FIRE_CODE                      object 
 14  FIRE_NAME                      object 
 15  ICS_209_PLUS_INCIDENT_JOIN_ID  object 
 16  ICS_209_PLUS_COMPLEX_JOIN_ID   object 
 17  MTBS_ID                        object 

In [6]:
#drop most of the columns
gdf = gdf[['FIRE_NAME', 'FIRE_YEAR', 
         'NWCG_CAUSE_CLASSIFICATION','NWCG_GENERAL_CAUSE',
         'FIRE_SIZE', 'LATITUDE',
         'LONGITUDE', 'STATE','FIPS_NAME']]

x = len(gdf)
print(gdf.shape)
print(gdf.dtypes)

gdf.tail()

(2303566, 9)
FIRE_NAME                     object
FIRE_YEAR                      int64
NWCG_CAUSE_CLASSIFICATION     object
NWCG_GENERAL_CAUSE            object
FIRE_SIZE                    float64
LATITUDE                     float64
LONGITUDE                    float64
STATE                         object
FIPS_NAME                     object
dtype: object


,FIRE_NAME,FIRE_YEAR,NWCG_CAUSE_CLASSIFICATION,NWCG_GENERAL_CAUSE,FIRE_SIZE,LATITUDE,LONGITUDE,STATE,FIPS_NAME
2303561,JONES,2020,Natural,Natural,1.0,39.037890,-108.959500,CO,None
2303562,POWER,2020,Missing data/not specified/undetermined,Missing data/not specified/undetermined,100.0,37.148611,-119.503056,CA,Madera County
2303563,12 MILE,2020,Natural,Natural,50.0,46.151370,-114.442800,MT,None
2303564,TAYLOR POND,2020,Natural,Natural,24892.0,46.670340,-120.114500,WA,Yakima County
2303565,MIDDLE MOUNTAIN,2020,Human,Missing data/not specified/undetermined,105.0,38.578900,-79.148450,VA,Rockingham County


In [7]:
gdf.STATE.unique()

array(['CA', 'NM', 'OR', 'NC', 'WY', 'CO', 'WA', 'MT', 'UT', 'AZ', 'SD',
       'AR', 'NV', 'ID', 'MN', 'TX', 'FL', 'SC', 'LA', 'OK', 'KS', 'MO',
       'NE', 'MI', 'KY', 'OH', 'IN', 'VA', 'IL', 'TN', 'GA', 'AK', 'ND',
       'WV', 'WI', 'AL', 'NH', 'PA', 'MS', 'ME', 'VT', 'NY', 'IA', 'DC',
       'MD', 'CT', 'MA', 'NJ', 'HI', 'DE', 'PR', 'RI'], dtype=object)

In [8]:
gdf.FIRE_YEAR.unique()

array([2005, 2004, 2006, 2008, 2002, 2007, 2009, 2001, 2003, 1992, 1993,
       1994, 1995, 1996, 1997, 1998, 1999, 2000, 2010, 2011, 2012, 2013,
       2014, 2015, 2016, 2017, 2018, 2019, 2020])

In [9]:
# Narrow down the STATE to CA
CA_gdf = gdf.loc[df['STATE'] == 'CA']

CA_gdf.head()

,FIRE_NAME,FIRE_YEAR,NWCG_CAUSE_CLASSIFICATION,NWCG_GENERAL_CAUSE,FIRE_SIZE,LATITUDE,LONGITUDE,STATE,FIPS_NAME
0,FOUNTAIN,2005,Human,Power generation/transmission/distribution,0.10,40.036944,-121.005833,CA,Plumas County
1,PIGEON,2004,Natural,Natural,0.25,38.933056,-120.404444,CA,Placer County
2,SLACK,2004,Human,Debris and open burning,0.10,38.984167,-120.735556,CA,El Dorado County
3,DEER,2004,Natural,Natural,0.10,38.559167,-119.913333,CA,Alpine County
4,STEVENOT,2004,Natural,Natural,0.10,38.559167,-119.933056,CA,Alpine County


In [10]:
CA_gdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 251881 entries, 0 to 2303562
Data columns (total 9 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   FIRE_NAME                  204514 non-null  object 
 1   FIRE_YEAR                  251881 non-null  int64  
 2   NWCG_CAUSE_CLASSIFICATION  251881 non-null  object 
 3   NWCG_GENERAL_CAUSE         251881 non-null  object 
 4   FIRE_SIZE                  251881 non-null  float64
 5   LATITUDE                   251881 non-null  float64
 6   LONGITUDE                  251881 non-null  float64
 7   STATE                      251881 non-null  object 
 8   FIPS_NAME                  157156 non-null  object 
dtypes: float64(3), int64(1), object(5)
memory usage: 19.2+ MB


In [11]:
# Narrow down the dataframe from any year equal to or greater than 2000
CA2_gdf = CA_gdf.loc[df['FIRE_YEAR'] >= 2000]

CA2_gdf.head()

,FIRE_NAME,FIRE_YEAR,NWCG_CAUSE_CLASSIFICATION,NWCG_GENERAL_CAUSE,FIRE_SIZE,LATITUDE,LONGITUDE,STATE,FIPS_NAME
0,FOUNTAIN,2005,Human,Power generation/transmission/distribution,0.10,40.036944,-121.005833,CA,Plumas County
1,PIGEON,2004,Natural,Natural,0.25,38.933056,-120.404444,CA,Placer County
2,SLACK,2004,Human,Debris and open burning,0.10,38.984167,-120.735556,CA,El Dorado County
3,DEER,2004,Natural,Natural,0.10,38.559167,-119.913333,CA,Alpine County
4,STEVENOT,2004,Natural,Natural,0.10,38.559167,-119.933056,CA,Alpine County


In [12]:
CA2_gdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 183884 entries, 0 to 2303562
Data columns (total 9 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   FIRE_NAME                  140261 non-null  object 
 1   FIRE_YEAR                  183884 non-null  int64  
 2   NWCG_CAUSE_CLASSIFICATION  183884 non-null  object 
 3   NWCG_GENERAL_CAUSE         183884 non-null  object 
 4   FIRE_SIZE                  183884 non-null  float64
 5   LATITUDE                   183884 non-null  float64
 6   LONGITUDE                  183884 non-null  float64
 7   STATE                      183884 non-null  object 
 8   FIPS_NAME                  154064 non-null  object 
dtypes: float64(3), int64(1), object(5)
memory usage: 14.0+ MB


In [24]:
# Double check FIRE_SIZE is in acre format
#CA2_Size_gdf = CA2_gdf.loc[df['FIRE_SIZE'] >= 1]

#CA2_Size_gdf.head()

,FIRE_NAME,FIRE_YEAR,NWCG_CAUSE_CLASSIFICATION,NWCG_GENERAL_CAUSE,FIRE_SIZE,LATITUDE,LONGITUDE,STATE,FIPS_NAME
8,SHASTA,2005,Human,Debris and open burning,1.0,41.233611,-122.283333,CA,None
13,TIGER,2004,Human,Recreation and ceremony,6.0,38.433333,-120.510000,CA,Amador County
16,POWER,2004,Human,Equipment and vehicle use,16823.0,38.523333,-120.211667,CA,Amador County
17,FREDS,2004,Human,Power generation/transmission/distribution,7700.0,38.780000,-120.260000,CA,El Dorado County
109,SADDLE,2005,Human,Equipment and vehicle use,10.0,33.718889,-117.433611,CA,None


In [13]:
CA2_gdf.NWCG_GENERAL_CAUSE.unique()

array(['Power generation/transmission/distribution', 'Natural',
       'Debris and open burning',
       'Missing data/not specified/undetermined',
       'Recreation and ceremony', 'Equipment and vehicle use',
       'Other causes', 'Fireworks', 'Arson/incendiarism', 'Smoking',
       'Misuse of fire by a minor', 'Firearms and explosives use',
       'Railroad operations and maintenance'], dtype=object)

In [14]:
CA2_gdf.NWCG_CAUSE_CLASSIFICATION.unique()

array(['Human', 'Natural', 'Missing data/not specified/undetermined'],
      dtype=object)

In [15]:
CA2_gdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 183884 entries, 0 to 2303562
Data columns (total 9 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   FIRE_NAME                  140261 non-null  object 
 1   FIRE_YEAR                  183884 non-null  int64  
 2   NWCG_CAUSE_CLASSIFICATION  183884 non-null  object 
 3   NWCG_GENERAL_CAUSE         183884 non-null  object 
 4   FIRE_SIZE                  183884 non-null  float64
 5   LATITUDE                   183884 non-null  float64
 6   LONGITUDE                  183884 non-null  float64
 7   STATE                      183884 non-null  object 
 8   FIPS_NAME                  154064 non-null  object 
dtypes: float64(3), int64(1), object(5)
memory usage: 14.0+ MB


In [46]:
geometry = gpd.points_from_xy(CA2_gdf['LONGITUDE'], CA2_gdf['LATITUDE'])
Geo_CA2_gdf = gpd.GeoDataFrame(CA2_gdf, geometry=geometry)

In [47]:
geojson_output = 'wildfire_geo_CA2.geojson'
Geo_CA2_gdf.to_file(geojson_output, driver='GeoJSON')

In [18]:
CA2_sorted_gdf = CA2_gdf.sort_values(by='FIRE_SIZE', ascending=False).head(20)

CA2_sorted_gdf.head(20)

,FIRE_NAME,FIRE_YEAR,NWCG_CAUSE_CLASSIFICATION,NWCG_GENERAL_CAUSE,FIRE_SIZE,LATITUDE,LONGITUDE,STATE,FIPS_NAME
2284521,DOE,2020,Natural,Natural,589368.0,39.765255,-122.672914,CA,Glenn County
2050664,RANCH,2018,Human,Equipment and vehicle use,410203.0,39.238889,-123.108611,CA,Mendocino County
2284490,SCU LIGHTNING COMPLEX,2020,Missing data/not specified/undetermined,Missing data/not specified/undetermined,396624.0,37.423905,-121.567713,CA,Santa Clara County
2284298,CREEK,2020,Human,Missing data/not specified/undetermined,379895.0,37.201005,-119.271613,CA,Fresno County
2284564,HOPKINS,2020,Natural,Natural,328363.0,40.125375,-123.072714,CA,Trinity County
2284391,CLAREMONT,2020,Natural,Natural,318776.0,39.858275,-120.912013,CA,Plumas County
1571658,RUSH,2012,Natural,Natural,315578.8,40.615300,-120.115300,CA,Lassen County
2284588,HENNESSEY,2020,Natural,Natural,305651.2,38.503937,-122.337262,CA,Napa County
1888566,THOMAS,2017,Human,Missing data/not specified/undetermined,281893.0,34.425278,-119.060278,CA,Ventura County
163744,CEDAR,2003,Human,Recreation and ceremony,280059.0,32.985556,-116.727500,CA,None


In [26]:
csv_file= 'wildfire20.csv'
CA2_sorted_gdf.to_csv(csv_file,index=False) 

In [27]:
CA_gdf_all=CA_gdf
csv_out='wildfireall.csv'
CA_gdf_all.to_csv(csv_out,index=False)

In [28]:
CA_sort_gdf = CA_gdf.sort_values(by='FIRE_SIZE', ascending=False)
csv_output='sortedwildfireall.csv'
CA_sort_gdf.to_csv(csv_output,index=False)

In [29]:
all_US_df= gdf.sort_values(by='FIRE_SIZE', ascending=False)
csv_put='Allwildfire.csv'
all_US_df.to_csv(csv_put,index=False)